In [32]:
import pandas as pd
import pandas
import numpy as np
import matplotlib.pyplot as plt
import pywt
from numpy.fft import fft
from sklearn.model_selection import train_test_split
import scipy
import scipy.fftpack
from sklearn.decomposition import PCA
import pickle
import pickle_compat

pickle_compat.patch()

In [33]:
testSet = pd.read_csv("test.csv",header=None)


In [34]:
def createmealfeaturematrix(data):
    op_matrix = pd.DataFrame()
    # time between max and min glucose
    op_matrix["Time_bet_max_min"] = (
        data.iloc[:, 22:25].idxmin(axis=1) - data.iloc[:, 5:19].idxmax(axis=1)
    ) * 5
    
    # normalize glucose difference
    op_matrix["Glucose_Difference_normalized"] = (
        data.iloc[:, 5:19].max(axis=1) - data.iloc[:, 22:25].min(axis=1)
    ) / (data.iloc[:, 22:25].min(axis=1))

    # windowed mean
    if data.shape[1] > 24:
        for i in range(6, data.shape[1], 6):
            op_matrix["Mean_" + str(i - 6)] = data.iloc[
                :, i : i + 6
            ].mean(axis=1)
    else:
        for i in range(0, data.shape[1], 6):
            op_matrix["Mean_" + str(i)] = data.iloc[:, i : i + 6].mean(
                axis=1
            )
            
    #entropy
    """ def get_entropy(series):
        series_counts = series.value_counts()
        return scipy.stats.entropy(series_counts)
    
    op_matrix['Entropy'] = data.apply(lambda row: get_entropy(row), axis=1) """
    

    # cgm velocity
    velocityDF = pd.DataFrame()
    for i in range(0, data.shape[1] - 5):
        velocityDF["Vel_" + str(i)] = data.iloc[:, i + 5] - data.iloc[:, i]
    op_matrix["Window_Velocity_Max"] = velocityDF.max(axis=1, skipna=True)
    
    #1st and 2nd differential
    tm=data.iloc[:,22:25].idxmin(axis=1)
    maximum=data.iloc[:,5:19].idxmax(axis=1)
    list1=[]
    second_differential_data=[]
    standard_deviation=[]
    for i in range(len(data)):
        list1.append(np.diff(data.iloc[:,maximum[i]:tm[i]].iloc[i].tolist()).max())
        second_differential_data.append(np.diff(np.diff(data.iloc[:,maximum[i]:tm[i]].iloc[i].tolist())).max())
        standard_deviation.append(np.std(data.iloc[i]))
    op_matrix['1stDifferential']=list1
    op_matrix['2ndDifferential']=second_differential_data

    # fft_4
    def get_fft(row):
        cgmFFTValues = np.abs(fft(row))
        cgmFFTValues.sort()
        return np.flip(cgmFFTValues)[0:4]
    
    FFT = pd.DataFrame()
    FFT["FFT_Top2"] = data.apply(lambda row: get_fft(row), axis=1)
    FFT_updated = pd.DataFrame(
        FFT.FFT_Top2.tolist(), columns=["FFT_2", "FFT_3", "FFT_4","FFT_5"]
    )
    FFT_updated.head()
    for i in range(1,5):
        op_matrix["FFT_" + str(i + 1)] = FFT_updated["FFT_" + str(i + 1)]

    return op_matrix

In [35]:
test_feature_matrix=pd.DataFrame()
test_feature_matrix=createmealfeaturematrix(testSet)

In [36]:
test_feature_matrix

,Time_bet_max_min,Glucose_Difference_normalized,Mean_0,Mean_6,Mean_12,Mean_18,Window_Velocity_Max,1stDifferential,2ndDifferential,FFT_2,FFT_3,FFT_4,FFT_5
0,45,0.910448,58.666667,84.333333,120.166667,83.333333,52,0,17,2079.0,359.604727,359.604727,147.498367
1,45,0.079545,96.666667,223.333333,283.833333,270.166667,159,2,8,5244.0,1180.074046,1180.074046,489.010200
2,90,1.785714,286.000000,280.333333,194.000000,135.166667,48,0,7,5373.0,1028.868002,1028.868002,284.385528
3,20,-0.014815,69.833333,76.500000,119.333333,136.500000,49,3,-1,2413.0,439.928157,439.928157,129.937271
4,25,0.225641,178.333333,182.500000,210.166667,215.500000,46,-6,4,4719.0,308.666625,308.666625,110.584192
5,85,0.431250,218.000000,221.333333,190.333333,163.833333,21,0,6,4761.0,381.275480,381.275480,101.986544
6,50,2.625000,69.000000,100.333333,124.666667,47.833333,56,1,22,2051.0,480.249620,480.249620,247.553857
7,85,0.020408,159.333333,139.833333,139.666667,137.833333,24,10,10,3460.0,130.878144,130.878144,119.057832
8,25,-0.087542,137.833333,158.333333,215.666667,293.000000,98,16,3,4829.0,894.081555,894.081555,422.247416
9,30,0.073944,155.500000,182.333333,279.333333,294.333333,98,0,4,5469.0,988.283102,988.283102,189.086172


In [39]:
pca = PCA(n_components=10)

principalComponents = pca.fit(test_feature_matrix)

testData = pca.fit_transform(test_feature_matrix)


In [40]:
clf = pickle.load(open('model.pkl', 'rb'))
y_pred = clf.predict(testData)
new_y_pred = []
for i in y_pred:
    if(i > 0):
        new_y_pred.append(1)
    else:
        new_y_pred.append(0)
        
result = pd.DataFrame(new_y_pred)
result.to_csv('Results.csv', index=False)